In [1]:
!pip install kaggle


In [2]:
import os
import json
from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
import pandas as pd

try:
    df = pd.read_csv('/content/IMDB Dataset.csv', engine='python')
except pd.errors.ParserError as e:
    print("ParserError: ", e)

# If you want to skip bad lines
try:
    df = pd.read_csv('/content/IMDB Dataset.csv', on_bad_lines='skip', engine='python')
except pd.errors.ParserError as e:
    print("ParserError: ", e)


ParserError:  unexpected end of data


In [4]:
df.head(5)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
df.shape

(7088, 2)

In [6]:
df['sentiment'].value_counts()

sentiment
negative    3547
positive    3541
Name: count, dtype: int64

In [7]:
df.replace({"sentiment":{"positive":1,"negative":0}},inplace=True)

In [8]:
df.head(5)

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [9]:
X = df['review']
Y = df['sentiment']

In [10]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=2)

In [12]:
print(X_train.shape,X_test.shape)

(5670, 100) (1418, 100)


In [11]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)
tokenizer.fit_on_texts(X_test)
X_train = pad_sequences(tokenizer.texts_to_sequences(X_train),maxlen=100)
X_test = pad_sequences(tokenizer.texts_to_sequences(X_test),maxlen=100)

In [13]:
print(X_train)

[[   4  266   85 ...    9  461  156]
 [  46   13  233 ...    4  487  178]
 [   5    1  299 ...    5  148 1848]
 ...
 [  10   65  510 ...    1 3073    5]
 [ 173    2   66 ...   27    4   94]
 [  20    1 1664 ...   23   16    9]]


In [14]:
print(X_test)

[[   0    0    0 ...  666 1527  168]
 [1585    8    1 ... 2455   53  151]
 [  60  150    2 ...    3 1275  321]
 ...
 [   0    0    0 ... 1293  902   15]
 [   0    0    0 ...  181  682   57]
 [  12   91  136 ... 1410  800  555]]


In [20]:
model = Sequential()
model.add(Embedding(5000,128,input_length=100))
model.add(LSTM(128,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1,activation='sigmoid'))

In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 128)          640000    
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 771713 (2.94 MB)
Trainable params: 771713 (2.94 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [25]:
model.fit(X_train,y_train,epochs=5,batch_size=64,validation_split=0.2)

Epoch 1/5
71/71 [==============================] - 39s 541ms/step - loss: 0.0584 - accuracy: 0.9797 - val_loss: 0.6354 - val_accuracy: 0.8086
Epoch 2/5
71/71 [==============================] - 30s 427ms/step - loss: 0.0463 - accuracy: 0.9852 - val_loss: 0.6943 - val_accuracy: 0.8104
Epoch 3/5
71/71 [==============================] - 31s 436ms/step - loss: 0.0531 - accuracy: 0.9835 - val_loss: 0.6065 - val_accuracy: 0.8157
Epoch 4/5
71/71 [==============================] - 31s 437ms/step - loss: 0.0826 - accuracy: 0.9696 - val_loss: 0.6773 - val_accuracy: 0.8095
Epoch 5/5
71/71 [==============================] - 31s 431ms/step - loss: 0.0237 - accuracy: 0.9918 - val_loss: 0.9064 - val_accuracy: 0.8183


In [27]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Validation Accuracy: {accuracy:.2f}')


45/45 [==============================] - 2s 40ms/step - loss: 0.9568 - accuracy: 0.7976
Validation Accuracy: 0.80
